In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule
import numpy as np

In [ ]:
data = torch.load('./pattern-beauty-50.pth')
print(len(data))

In [ ]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
from sequential.seq2pat import Seq2Pat, Attribute

# data = [
#     [1,2,112,2223,3,134],
#     [1,2,6456,324,1521345,123],
#     [1,2,125,4365,23421,203985],
#     [2,3,12358,4532,23541,38954],
#     [2,3,353,4234,45453,4678],
#     [2,3,234,3342,11234,465],
# ]

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=data, n_jobs=2, max_span=5)
# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=2)
patterns_value = [_[:-1] for _ in patterns]
patterns_count = [_[-1] for _ in patterns]
# print(patterns)
print(len(patterns))

In [ ]:
# Remove duplicate pattern
if False:
    from itertools import combinations
    result = []
    help_set = set()
    patterns_value.sort(key=len, reverse=True)
    for p in patterns_value:
        if tuple(p) in help_set:
            continue
        else:
            result.append(p)
            for r in range(2, len(p) + 1):
                for _ in combinations(p, r):
                    help_set.add(tuple(_))
    patterns_value_new = result
    print(len(patterns_value_new))
    patterns_value = patterns_value_new

In [ ]:
original_train = torch.load('./dataset/amazon-beauty-50/beauty/train_ori.pth')
len(original_train)

In [ ]:
max_seq_len = 50
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:]
    else:
        return seq + [0] * (max_seq_len - cur_seq_len)

train_set = set()

for pattern in patterns_value:
    seq = pattern
    seq_len = len(seq)
    train_set.add(tuple(
        truncate_or_pad(seq[:-1]) + truncate_or_pad(seq[1:])
    ))

In [ ]:
train_list = []
for _ in list(train_set):
    train_item_seq = _[:max_seq_len]
    target_item_seq = _[max_seq_len:]
    seq_len = sum([a != 0 for a in train_item_seq])
    train_list.append([
        0,
        train_item_seq,
        target_item_seq,
        seq_len,
        [1] * seq_len + [0] * (max_seq_len - seq_len),
        [0] * max_seq_len,
    ])
print(len(train_list))
print(len(original_train))

In [ ]:
torch.save(train_list + original_train, 'train_new2-3.pth')

In [ ]:
torch.save(train_list, 'train_new-pure2-3.pth')